# Dijkstra from scratch
https://joshuacclark.medium.com/dijkstras-algorithm-with-adjacency-lists-ae2e9301d315

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict 

import heapq

from sklearn.preprocessing import StandardScaler

from scipy.special import betainc

from concurrent.futures import ProcessPoolExecutor, as_completed

In [1]:
class SearchEdge:

    def __init__(self,adj_mat:np.ndarray):
        self.adj_list = self.adjacenty_from_matrix_to_list(adj_mat)
        self.n = len(self.adj_list)

    def adjacenty_from_matrix_to_list(self,adj_mat:np.ndarray) -> defaultdict:

        adj_list = defaultdict(list)

        for i,j in zip(*np.where(np.triu(adj_mat))):
            adj_list[i].append([j,adj_mat[i,j]])
            adj_list[j].append([i,adj_mat[j,i]])

        return adj_list

    def min_distance(self, dist:list, seen:set):

        min_dist = float('inf')

        for i in range(len(self.adj_list)):
            if i not in seen and dist[i] < min_dist: # Avoid to turn
                min_d = dist[i]
                min_node = i
                
        return min_node
                
    def slow_dijkstra(self,start):

        dist = [float('inf')]*len(self.adj_list)
        seen=set()
        dist[start]=0

        for _ in range(len(self.adj_list)):
            current_node = self.min_distance(dist, seen)
            seen.add(current_node)

            for node, weight in self.adj_list[current_node]:
                if node not in seen and dist[node] > dist[current_node] + weight:
                    dist[node] = dist[current_node] + weight

    def dijkstra_optim(self,start_node:int,end_node:int=-1,depth_lim=None):

        dist = [float('inf')] * self.n
        seen = set ()
        heap = []
        depth_tracker = [0] * self.n
        previous_nodes = [None] * self.n

        dist[start_node]=0
        heapq.heappush(heap, (0, start_node, 0))

        while heap:
            current_dist, node, current_depth = heapq.heappop(heap)

        pass

    def get_path(self,previous_node,start_node,end_node):

        path = []
        node = end

        with node is not None:
            path.insert(0, node)
            if node == start_node:
                break
            node = previous_nodes[node]
        return path if path[0] == start_node else None
            
        
        

        

        

        

        
        

    

        
                

SyntaxError: incomplete input (521537507.py, line 50)

In [79]:
matrix = np.array([[0, 7, 9, 0, 0, 14],
                    [7, 0, 10, 15, 0, 0],
                    [9, 10, 0, 11, 0, 2],
                    [0, 15, 11, 0, 6, 0],
                    [0, 0, 0, 6, 0, 9],
                    [14, 0, 2, 0, 9, 0]])

In [80]:
se = SearchEdge(matrix)
se.slow_dijkstra(start=0)

[0, 7, 9, 29, 23, 14]


In [56]:
test = adjacenty_from_matrix_to_list(adj_mat=matrix)

In [67]:
test

defaultdict(list,
            {0: [[1, 7], [2, 9], [5, 14]],
             1: [[0, 7], [2, 10], [3, 15]],
             2: [[0, 9], [1, 10], [3, 11], [5, 2]],
             5: [[0, 14], [2, 2], [4, 9]],
             3: [[1, 15], [2, 11], [4, 6]],
             4: [[3, 6], [5, 9]]})

In [7]:
def dijkstra_depth_lim(adj_mat,start,end):

    #init priority queue
    priority_queue = [(0,start,0)]

    while priority_queue:
        current_dist, current_node, current_depth = heapq.heappop(priority_queue)

        print(current_dist)
        print(current_node)
        print(current_depth)

In [8]:
dijkstra_depth_lim(adj_mat=matrix,start=0,end=2)

0
0
0


In [83]:
import heapq

def dijkstra_with_target_and_depth(self, source, end, depth_limit):
    dist = [float('inf')] * self.V  # Distance to each node
    seen = set()  # Visited nodes
    heap = []  # Min-heap priority queue
    depth_tracker = [0] * self.V  # Track the depth of each node
    previous_nodes = [None] * self.V  # To reconstruct the path

    dist[source] = 0
    heapq.heappush(heap, (0, source, 0))  # (distance, node, depth)

    while heap:
        current_dist, node, current_depth = heapq.heappop(heap)

        # If depth exceeds limit, skip this path
        if current_depth > depth_limit:
            continue

        # If the target node is reached, stop
        if node == end:
            return dist[end], self.reconstruct_path(previous_nodes, source, end)

        # Mark the node as seen
        seen.add(node)

        # Explore neighbors
        for conn, weight in self.graph[node]:
            if conn not in seen:
                new_dist = current_dist + weight
                new_depth = current_depth + 1

                # Update if a shorter path is found
                if new_dist < dist[conn]:
                    dist[conn] = new_dist
                    depth_tracker[conn] = new_depth
                    previous_nodes[conn] = node
                    heapq.heappush(heap, (new_dist, conn, new_depth))

    # If the target node is not reachable within the depth limit
    return float('inf'), None

def reconstruct_path(self, previous_nodes, source, target):
    path = []
    current = target
    while current is not None:
        path.insert(0, current)
        if current == source:
            break
        current = previous_nodes[current]
    return path if path[0] == source else None
